# PyMem3DG Tutorial 7 - Extended Applications 3: membrane mechanochemical dynamics
`Cuncheng Zhu, Christopher T. Lee`

This tutorial covers how `Mem3DG` models endocytic budding on spherical membrane using protein dynamics coupled with membrane mechanics. 

In [16]:
import pymem3dg as dg
import numpy as np
import pymem3dg.visual as dg_vis
import polyscope as ps
import pymem3dg.read as dg_read

CONTINUE = False # change this to True if continue from an existing trajectory file 
outputDir = "output/tutorial7/traj/"
trajFile = outputDir + "traj.nc"
face, vertex = dg.getIcosphere(radius=1, subdivision=3)
if CONTINUE:
    continuationFrame = dg_read.sizeOf(trajnc=trajFile) - 1
else:
    continuationFrame = 0


In [17]:
p = dg.Parameters()

p.proteinMobility = 3
p.temperature = 0

p.variation.isProteinVariation = True
p.variation.isShapeVariation = True

p.bending.Kb = 8.22e-5
p.bending.Kbc = 8.22e-5
p.bending.H0c = 10

p.tension.Ksg = 1
p.tension.At = 12.5025

p.adsorption.epsilon = -1e-3

p.osmotic.isPreferredVolume = True
p.osmotic.Kv = 0.05
p.osmotic.Vt = 0.7 * 4.15889

p.dirichlet.eta = p.bending.Kb

In [6]:
proteinDensity = np.ones(np.shape(vertex)[0]) * 0.1
velocity = np.zeros(np.shape(vertex))

if CONTINUE:
    g = dg.System(trajFile=trajFile, startingFrame=continuationFrame, parameters=p)
else:
    g = dg.System(
        topologyMatrix=face,
        vertexMatrix=vertex,
        proteinDensity=proteinDensity,
        velocity=velocity,
        parameters=p,
    )


In [7]:
g.meshProcessor.meshMutator.isShiftVertex = True
g.meshProcessor.meshMutator.flipNonDelaunay = True
# g.meshProcessor.meshMutator.splitLarge = True
g.meshProcessor.meshMutator.splitFat = True
g.meshProcessor.meshMutator.splitSkinnyDelaunay = True
g.meshProcessor.meshMutator.splitCurved = True
g.meshProcessor.meshMutator.curvTol = 0.004
g.meshProcessor.meshMutator.collapseSkinny = True
# g.meshProcessor.meshMutator.collapseSmall = True
# g.meshProcessor.meshMutator.collapseFlat = True
# g.meshProcessor.meshMutator.targetFaceArea = 0.0003 * R_bar**2
g.meshProcessor.meshMutator.isSmoothenMesh = True

In [8]:
g.initialize(nMutation=0, ifMute=False)

area_init = 12.5065
vol_init = 4.15274
Characteristic volume wrt to At = 4.1569


In [ ]:
""" Integrator construction """
fe = dg.Euler(
    system=g,
    characteristicTimeStep=0.25,
    totalTime=20000,
    savePeriod=1000 * 0.25,
    tolerance=0,
    outputDirectory=outputDir,
    frame=continuationFrame,
)
""" settings """
# fe.updateGeodesicsPeriod = 100
fe.processMeshPeriod = 10000
# fe.fluctuatePeriod = 10
# fe.fluctuateAmplitude = 0.001
fe.isBacktrack = True
# fe.ifAdaptiveStep = False
""" Verbosity """
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
# fe.ifOutputMeshFile = True

fe.integrate()


: 

: 

In [20]:

dg_vis.animate(trajNc=trajFile, parameters = p)
ps.show()